In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Modules required**

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import os

# to disable all debugging logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

**Importing Data Set**

In [ ]:
dir = os.listdir('../input/plant-village/PlantVillage')
# for filenames in dir:
#     print(filenames)

**Extracting Potato Dataset from PlantVillage**

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_Leaf_Mold ./Tomato_Leaf_Mold

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus ./Tomato__Tomato_YellowLeaf__Curl_Virus

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_Bacterial_spot ./Tomato_Bacterial_spot

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_healthy ./Tomato_healthy

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_Spider_mites_Two_spotted_spider_mite ./Tomato_Spider_mites_Two_spotted_spider_mite

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_Early_blight ./Tomato_Early_blight

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato__Target_Spot ./Tomato__Target_Spot

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato_Late_blight ./Tomato_Late_blight

In [ ]:
!cp -rf ../input/plant-village/PlantVillage/Tomato__Tomato_mosaic_virus ./Tomato__Tomato_mosaic_virus

**Engage the Dataset in Kaggle/Working Dir.**

In [ ]:
Current_Dir = os.getcwd()
dataset_dir = pathlib.Path(Current_Dir)
print(dataset_dir)

**#Global initialization of variables**

In [ ]:
Image_Size = 150
Batch_Size = 50
Channels = 3
EPOCHS = 50

**Pipeline Initialization**

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
dataset_dir,
shuffle=True,
image_size=(Image_Size, Image_Size),
batch_size=Batch_Size
)

**Class Names Initialization**

In [ ]:
class_names=dataset.class_names
class_names

**Explore Dataset**

In [ ]:
len(dataset)

In [ ]:
# Exploring images in a single Batch
for images, labels in dataset.take(1):
    print(images[0].numpy())
    print(images[0].shape)
    print(labels.numpy())    

In [ ]:
# Explore an Image and its Label
for images, labels in dataset.take(1):
    plt.imshow(images[0].numpy().astype('uint8'))
    plt.axis('off')
    plt.title(class_names[labels[0].numpy()])

In [ ]:
# Exploring Multiple Images
plt.figure(figsize=(12,12))
for image_batch, label_batch in dataset.take(1):
    for i in range(15):
        plt.subplot(5,3,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.axis('off')
        plt.title(class_names[label_batch[i]])

**Spliting Dataset for Train , Test and Validation**

In [ ]:
# Function for Splitting the data
def split_dataset(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 10)
        
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train, val, test = split_dataset(dataset)
len(train),len(val),len(test)

**Caching,Shuffling and Prefetching the data for training Optimization**

In [ ]:
train = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val = val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

**Data Optimization**

In [ ]:
# Image Preprocessing Process: Rescaling and Resizing
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(Image_Size, Image_Size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
# Data augmentation 
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(factor = 0.2)
])

**Model Building**

In [ ]:
input_shape=(Image_Size, Image_Size, Channels)
n_classes=9
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False ## Not trainable weights

base_model.summary()

****ANN Layer****

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(9, activation='softmax')

**Final Model**

In [ ]:
model = models.Sequential([
    resize_and_rescale, 
    data_augmentation,
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
vgghistory = model.fit(
    train, 
    epochs = EPOCHS, 
    batch_size = Batch_Size, 
    verbose = 1, 
    validation_data = val,
    callbacks=[early_stopping])